In [3]:
root_dir = "output"
method_name = "standard"
benchmark = "tabmwp"

model = "gpt-3.5-turbo"
eval_model = "gpt-4o-mini"
seed = 42
num_retries = 1
warming = [0.0]

In [4]:
from agential.core.llm import LLM
import os
llm = LLM(
        model, 
        api_key = api_key,
        temperature=0,
        top_p=1,
        frequency_penalty=0.0,
        presence_penalty=0.0,
        seed=seed
)
eval_llm = LLM(
    eval_model,
    api_key = api_key,
    temperature=0,
    top_p=1,
    frequency_penalty=0.0,
    presence_penalty=0.0,
    seed=seed
)

In [5]:
from agential.prompting.standard.prompting import Standard
method = Standard(
    llm=llm,
    benchmark=benchmark,
)

In [6]:
from datasets import load_dataset

/opt/anaconda3/envs/agential/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
data = load_dataset("google-research-datasets/mbpp")['test']

NameError: name 'load_dataset' is not defined

In [10]:
from agential.eval.metrics.classification import EM


EM('Done', "Done", normalize=False)

True

In [13]:
from agential.eval.metrics.classification import EM
from agential.utils.general import safe_execute

eval_table_data = []
perf_table_data = []
em_scores = []
outputs = []
index = 0 
for instance in data:
    if index > 5:
        break
    index += 1
    question = instance['text']
    answer = instance['code']
    seq = '\n'.join(instance['test_list'])
    
    # Building the prompt
    question = f"""You are an expert Python programmer, and here is your task:
Write a function to find the shared elements from the given two lists.
Your code should pass these tests:

{seq}"""
    
    print("-----------------------------------")
    print(question)

    print("\n"+answer)
    print("-----------------------------------")
    # Inference
    out = method.generate(
        question=question,
        key=answer,
        num_retries=num_retries,
        warming=warming
    )


    print("OUTPUT")
    print(out.answer)
    print("-----------------------------------")
    code_str = out.answer.replace("```python", "").replace("```", "").strip()
    _, execution_status = safe_execute(
        f"from typing import *\n\n{code_str}\n{answer}"
    )
    print("EXECUTION STATUS")
    print(execution_status)
    print("-----------------------------------")

    is_correct = int(EM(execution_status, "Done", normalize=False))


    # Update scores.
    em_scores.append(is_correct)
    # Update tables.
    eval_table_data.append([question, answer, execution_status, out.answer, is_correct])
    perf_table_data.append([
        out.total_prompt_tokens, 
        out.total_completion_tokens, 
        out.total_tokens, 
        out.total_prompt_cost,
        out.total_completion_cost,
        out.total_cost,
        out.total_prompt_time,
        out.total_time
    ])
    # Update outputs.
    outputs.append(out)

    print("XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")
    
    

-----------------------------------
You are an expert Python programmer, and here is your task:
Write a function to find the shared elements from the given two lists.
Your code should pass these tests:

assert remove_Occ("hello","l") == "heo"
assert remove_Occ("abcda","a") == "bcd"
assert remove_Occ("PHP","P") == "H"

def remove_Occ(s,ch): 
    for i in range(len(s)): 
        if (s[i] == ch): 
            s = s[0 : i] + s[i + 1:] 
            break
    for i in range(len(s) - 1,-1,-1):  
        if (s[i] == ch): 
            s = s[0 : i] + s[i + 1:] 
            break
    return s 
-----------------------------------
OUTPUT

```python
def remove_Occ(s, c):
    return s.replace(c, '')

assert remove_Occ("hello", "l") == "heo"
assert remove_Occ("abcda", "a") == "bcd"
assert remove_Occ("PHP", "P") == "H"
```

-----------------------------------
EXECUTION STATUS
Done
-----------------------------------
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
-----------------------------------


In [14]:
em_scores

[1, 0, 0, 0, 0, 1]